<a href="https://colab.research.google.com/github/cundeyu154/PL-Repo/blob/main/HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread pandas google-auth google-auth-oauthlib gradio matplotlib seaborn
print("函式庫安裝完成。")

函式庫安裝完成。


In [ ]:
# Cell 2: Google Drive 掛載與路徑設定
from google.colab import drive
import os

# 1. 掛載 Google Drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Drive 掛載失敗: {e}")

# 2. 設定您的專案路徑和 Google Sheet 名稱
# 請確保您在 Drive 中建立了一個名為 'Task_Tracker_Project' 的資料夾，
# 並將 'service_account.json' 檔案放在裡面。
PROJECT_FOLDER_NAME = 'Task_Tracker_Project' # 你的專案資料夾名稱
PROJECT_PATH = f'/content/drive/MyDrive/{PROJECT_FOLDER_NAME}/'

# 替換為你在步驟 1 建立的 Google Sheet 名稱
SPREADSHEET_NAME = 'My Task Tracker'
WORKSHEET_NAME = '工作表1'
SERVICE_ACCOUNT_FILE = PROJECT_PATH + 'service_account.json'

# 3. 建立專案資料夾 (如果不存在)
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)
    print(f"已在 Google Drive 中建立資料夾: {PROJECT_PATH}")

print(f"專案路徑: {PROJECT_PATH}")
print(f"試算表名稱: {SPREADSHEET_NAME}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
專案路徑: /content/drive/MyDrive/Task_Tracker_Project/
試算表名稱: My Task Tracker


In [ ]:
# Cell 3: Google Sheet 管理器 (gs_manager.py 內容)

import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
from datetime import datetime

# 使用 Cell 2 中設定的全域變數
SCOPE = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

class SheetManager:
    def __init__(self):
        try:
            # 認證
            creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPE)
            client = gspread.authorize(creds)
            # 開啟試算表
            self.spreadsheet = client.open(SPREADSHEET_NAME)
            self.worksheet = self.spreadsheet.worksheet(WORKSHEET_NAME)
            print("Google Sheet 連線成功。")
        except Exception as e:
            print(f"🚨 Google Sheet 連線失敗，請檢查 service_account.json、共用設定或試算表名稱。錯誤: {e}")
            self.worksheet = None

    def fetch_all_tasks(self):
        """從 Google Sheet 讀取所有資料，並以 list of dicts 格式回傳。"""
        if not self.worksheet: return []
        # get_all_records() 會將第一行視為欄位名稱
        data = self.worksheet.get_all_records()
        return data

    def upload_data(self, data_list):
        """將 list of dicts 資料寫入 Google Sheet (覆寫)。"""
        if not self.worksheet: return
        if not data_list:
            # 只保留標頭
            headers = ['ID', '任務名稱', '描述', '建立時間', '預計完成時間', '狀態']
            self.worksheet.clear()
            self.worksheet.append_rows([headers], value_input_option='USER_ENTERED')
            return

        headers = list(data_list[0].keys())
        values = [list(d.values()) for d in data_list]

        self.worksheet.clear()
        self.worksheet.append_rows([headers] + values, value_input_option='USER_ENTERED')
        print("✅ 資料已成功上傳至 Google Sheet。")

    # 【✅ 5. 新增功能：「刪除任務」】
    def delete_task_by_id(self, task_id):
        """根據 ID 刪除 Google Sheet 中的一列資料。"""
        if not self.worksheet: return False

        all_data = self.worksheet.get_all_values()
        if not all_data: return False

        header = all_data[0]
        data_rows = all_data[1:]

        try:
            id_index = header.index('ID')
        except ValueError:
            print("Google Sheet 缺少 'ID' 欄位。")
            return False

        row_to_delete_index = -1
        for i, row in enumerate(data_rows):
            # 比較字串以適應 gspread 讀取的格式
            if str(row[id_index]).strip() == str(task_id):
                # +2 是因為 Google Sheet 的行號從 1 開始，且要加上標頭行 (index 0)
                row_to_delete_index = i + 2
                break

        if row_to_delete_index != -1:
            self.worksheet.delete_rows(row_to_delete_index)
            return True
        else:
            return False

    # 【✅ 6. 查詢強化：完成任務、時間範圍查詢】
    def query_tasks(self, query_type='all', start_date=None, end_date=None):
        """根據狀態或時間範圍查詢任務。"""
        tasks = self.fetch_all_tasks()

        # 狀態過濾
        if query_type == 'completed':
            tasks = [t for t in tasks if str(t.get('狀態', '')).strip() == '完成']
        elif query_type == 'pending':
            tasks = [t for t in tasks if str(t.get('狀態', '')).strip() == '待辦']

        # 時間範圍過濾 (假設 '建立時間' 欄位為 YYYY-MM-DD HH:MM:SS 格式)
        if start_date or end_date:
            filtered_tasks = []
            try:
                start_dt = datetime.strptime(start_date, '%Y-%m-%d') if start_date else None
                end_dt = datetime.strptime(end_date, '%Y-%m-%d') if end_date else None
            except ValueError:
                return [] # 日期格式錯誤則不進行查詢

            for task in tasks:
                try:
                    # 只取日期部分進行比較
                    task_date_str = task.get('建立時間', '').split()[0]
                    task_dt = datetime.strptime(task_date_str, '%Y-%m-%d')

                    is_in_range = True
                    if start_dt and task_dt < start_dt:
                        is_in_range = False
                    # 注意：查詢結束日包含當天，所以通常是 <= end_dt
                    if end_dt and task_dt > end_dt:
                        is_in_range = False

                    if is_in_range:
                        filtered_tasks.append(task)
                except:
                    # 忽略格式不正確的日期
                    continue
            tasks = filtered_tasks

        return tasks

In [ ]:
# Cell 4: 任務資料管理器 (task_data_manager.py 內容)

# 依賴 Cell 3 中的 SheetManager
# 依賴 Cell 2 中的 PROJECT_PATH

class TaskDataManager:
    def __init__(self):
        self.sheet_manager = SheetManager()
        # 核心資料結構: list of dicts (✅ 2)
        self.tasks = self.load_tasks_from_sheet()

    def load_tasks_from_sheet(self):
        """從 Google Sheet 載入資料。"""
        return self.sheet_manager.fetch_all_tasks()

    def add_task(self, name, description, est_complete_date):
        """新增任務並更新 Google Sheet。"""
        # 重新載入確保 ID 生成基於最新資料
        self.tasks = self.load_tasks_from_sheet()
        new_id = self._generate_next_id()
        new_task = {
            'ID': new_id,
            '任務名稱': name,
            '描述': description,
            '建立時間': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            '預計完成時間': est_complete_date,
            '狀態': '待辦'
        }
        self.tasks.append(new_task)
        self.sheet_manager.upload_data(self.tasks)
        return new_task

    def toggle_task_status(self, task_id, new_status):
        """切換任務狀態 (待辦 <-> 完成) 並更新 Google Sheet。"""
        self.tasks = self.load_tasks_from_sheet()
        task_id = int(task_id)

        found = False
        for task in self.tasks:
            if task.get('ID') == task_id:
                task['狀態'] = new_status
                found = True
                break

        if found:
            self.sheet_manager.upload_data(self.tasks) # 重新上傳更新
            return True
        return False

    def _generate_next_id(self):
        """產生一個新的、不重複的 ID。"""
        if not self.tasks:
            return 1
        # 確保所有 ID 都是數字以便 max() 運算
        max_id = 0
        for task in self.tasks:
            try:
                max_id = max(max_id, int(task.get('ID', 0)))
            except (TypeError, ValueError):
                continue # 忽略無效 ID
        return max_id + 1

    # 【✅ 3. 可以把結果匯出與匯入紀錄（CSV / JSON）】
    def export_data(self, file_path_relative, file_format='csv'):
        """將當前任務列表匯出為 CSV 或 JSON 檔案。"""
        self.tasks = self.load_tasks_from_sheet() # 確保匯出的是最新資料
        full_path = PROJECT_PATH + file_path_relative
        df = pd.DataFrame(self.tasks)

        if file_format.lower() == 'csv':
            df.to_csv(full_path, index=False, encoding='utf-8')
        elif file_format.lower() == 'json':
            # orient='records' 得到 [{}, {}, ...] 格式
            df.to_json(full_path, orient='records', indent=4, force_ascii=False)
        else:
            return f"不支援的格式: {file_format}。"

        return f"✅ 資料成功匯出至 Google Drive: {full_path}。"

    def import_data(self, file_path_relative, file_format='csv'):
        """從 CSV 或 JSON 檔案匯入資料並更新 Google Sheet。"""
        full_path = PROJECT_PATH + file_path_relative

        if not os.path.exists(full_path):
            return f"🚨 檔案不存在: {full_path}"

        try:
            if file_format.lower() == 'csv':
                df = pd.read_csv(full_path)
            elif file_format.lower() == 'json':
                df = pd.read_json(full_path, orient='records')
            else:
                return f"🚨 不支援的匯入格式: {file_format}。"
        except Exception as e:
            return f"🚨 檔案讀取失敗，請檢查格式: {e}"

        # 轉換為 list of dicts
        imported_tasks = df.to_dict('records')

        # 覆蓋/更新當前任務
        self.tasks = imported_tasks
        self.sheet_manager.upload_data(self.tasks) # 更新 Google Sheet

        return f"✅ 成功從 {full_path} 匯入 {len(imported_tasks)} 筆任務，已更新 Google Sheet。"

    # 【✅ 5. 新增功能：「刪除任務」】
    def delete_task(self, task_id):
        """刪除任務並更新 Google Sheet。"""
        task_id = int(task_id)

        # 1. 嘗試從 Google Sheet 刪除單行 (更快)
        success = self.sheet_manager.delete_task_by_id(task_id)

        if success:
             # 2. 重新載入，確保本地資料同步
             self.tasks = self.load_tasks_from_sheet()
             return f"✅ 成功刪除 ID: {task_id} 的任務 (已更新 Google Sheet)。"
        else:
            # 如果單行刪除失敗 (例如找不到 ID)，可能是資料過期，再重新嘗試
            self.tasks = self.load_tasks_from_sheet()
            initial_length = len(self.tasks)
            self.tasks = [task for task in self.tasks if task.get('ID') != task_id]

            if len(self.tasks) < initial_length:
                # 找到並在本地刪除，現在上傳整個列表覆蓋
                self.sheet_manager.upload_data(self.tasks)
                return f"✅ 成功刪除 ID: {task_id} 的任務 (已更新 Google Sheet)。"
            else:
                return f"🚨 找不到 ID: {task_id} 的任務。"

In [ ]:
# Cell 5: 視覺化模組 (visualization.py 內容)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 依賴 Cell 2 中的 PROJECT_PATH

def generate_task_status_plot(tasks):
    """生成任務狀態（待辦/完成）的圓餅圖。"""
    if not tasks:
        return None # Gradio 期望 None 來清空 Image 輸出

    df = pd.DataFrame(tasks)
    # 確保只統計 '待辦' 和 '完成'
    df['狀態'] = df['狀態'].apply(lambda x: str(x).strip())
    status_counts = df[df['狀態'].isin(['待辦', '完成'])]['狀態'].value_counts()

    if status_counts.empty:
        return None

    # 設置中文字體以防亂碼 (Colab 環境)
    plt.rcParams['font.sans-serif'] = ['DFKai-SB', 'Arial Unicode MS']
    plt.rcParams['axes.unicode_minus'] = False

    # 創建圖表並儲存
    temp_file = PROJECT_PATH + "status_plot.png"

    plt.figure(figsize=(7, 7))
    plt.pie(
        status_counts,
        labels=status_counts.index,
        autopct='%1.1f%%',
        startangle=90,
        colors=sns.color_palette('pastel')
    )
    plt.title('任務狀態分佈', fontsize=16)
    plt.axis('equal')

    plt.savefig(temp_file)
    plt.close()

    print(f"✅ 統計圖已儲存至: {temp_file}")
    return temp_file # 回傳檔案路徑供 Gradio 顯示

In [ ]:
# 強制測試資料管理器初始化與讀取
data_manager = TaskDataManager()
if data_manager.tasks:
    print(f"✅ 成功讀取 {len(data_manager.tasks)} 筆任務。")
else:
    print("✅ 成功連線，但試算表為空（這沒關係）。")

Google Sheet 連線成功。
✅ 成功連線，但試算表為空（這沒關係）。


In [ ]:
# Cell 6: Gradio 應用介面 (app.py 內容)

import gradio as gr
# 依賴 Cell 3, 4, 5 中的類別和函式

# 初始化資料管理器
data_manager = TaskDataManager()

def display_tasks(tasks=None):
    """將任務列表轉換為 Gradio Dataframe 格式，並確保結構穩定。"""
    if tasks is None:
        tasks = data_manager.tasks

    # 定義預期的欄位，以防連線失敗或資料為空
    EXPECTED_COLUMNS = ['ID', '任務名稱', '描述', '建立時間', '預計完成時間', '狀態']

    if not tasks:
        # 如果資料為空，返回一個具有正確標頭的空 DataFrame
        return pd.DataFrame(columns=EXPECTED_COLUMNS)

    df = pd.DataFrame(tasks)

    # 確保 DataFrame 包含所有預期欄位
    for col in EXPECTED_COLUMNS:
        if col not in df.columns:
            df[col] = '' # 缺少則補上空欄位

    # 排序欄位以匹配預期順序
    df = df.reindex(columns=EXPECTED_COLUMNS)
    return df

# --- Gradio 核心處理函式 ---

def handle_add_task(name, description, est_complete_date):
    """新增任務的處理函式。"""
    if not name or not est_complete_date:
        return "🚨 錯誤：任務名稱和預計完成時間不能為空。", display_tasks()

    try:
        datetime.strptime(est_complete_date, '%Y-%m-%d')
    except ValueError:
        return "🚨 錯誤：日期格式必須是 YYYY-MM-DD。", display_tasks()

    data_manager.add_task(name, description, est_complete_date)
    return "✅ 任務新增成功！ (已同步至 Google Sheet)", display_tasks()

def handle_delete_task(task_id):
    """刪除任務的處理函式 (✅ 5)。"""
    try:
        task_id = int(task_id)
    except ValueError:
        return "🚨 錯誤：任務 ID 必須是數字。", display_tasks()

    message = data_manager.delete_task(task_id)
    return message, display_tasks()

def handle_toggle_status(task_id, new_status):
    """切換任務狀態的處理函式 (✅ 6.1)。"""
    try:
        task_id = int(task_id)
    except ValueError:
        return "🚨 錯誤：任務 ID 必須是數字。", display_tasks()

    success = data_manager.toggle_task_status(task_id, new_status)
    message = "✅ 狀態更新成功。 (已同步至 Google Sheet)" if success else "🚨 找不到該 ID 的任務。"
    return message, display_tasks()

# 【✅ 6. 查詢強化：完成任務、時間範圍查詢】
def handle_query_tasks(query_type, start_date, end_date):
    """處理任務查詢。"""

    # 清理輸入
    start_date = start_date.strip() if start_date else None
    end_date = end_date.strip() if end_date else None

    # 呼叫 SheetManager 中的查詢邏輯
    queried_tasks = data_manager.sheet_manager.query_tasks(
        query_type=query_type.lower(),
        start_date=start_date,
        end_date=end_date
    )

    return display_tasks(queried_tasks)

def handle_visualize():
    """視覺化處理函式 (✅ 4)。"""
    data_manager.tasks = data_manager.load_tasks_from_sheet() # 確保最新資料
    plot_path = generate_task_status_plot(data_manager.tasks)
    return plot_path

def handle_file_io(action, file_path_relative, file_format):
    """處理匯入/匯出檔案 I/O (✅ 3)。"""
    if action == "匯出":
        message = data_manager.export_data(file_path_relative, file_format)
        return message, display_tasks()
    elif action == "匯入":
        message = data_manager.import_data(file_path_relative, file_format)
        return message, display_tasks()
    return "🚨 請選擇匯入或匯出。", display_tasks()

def handle_reload():
    """重新載入最新資料"""
    data_manager.tasks = data_manager.load_tasks_from_sheet()
    return "✅ 資料已從 Google Sheet 重新載入。", display_tasks()

with gr.Blocks(title="任務追蹤器") as demo:
    gr.Markdown("# 🚀 Google Sheet 任務追蹤器 (Colab Demo)")

    output_status_message = gr.Textbox(label="操作訊息", value="歡迎使用！", interactive=False)

    # 定義表格預期的欄位標頭，以避免 Gradio 的內部推斷錯誤
    TABLE_HEADERS = ['ID', '任務名稱', '描述', '建立時間', '預計完成時間', '狀態']

    # 顯示任務列表
    # Cell 6: Gradio 應用介面 (app.py 內容)

import gradio as gr
# ... (其他程式碼保持不變) ...

# --- Gradio 介面定義 ---

with gr.Blocks(title="任務追蹤器") as demo:
    gr.Markdown("# 🚀 Google Sheet 任務追蹤器 (Colab Demo)")

    output_status_message = gr.Textbox(label="操作訊息", value="歡迎使用！", interactive=False)

    # 定義表格預期的欄位標頭
    TABLE_HEADERS = ['ID', '任務名稱', '描述', '建立時間', '預計完成時間', '狀態']


    task_table = gr.Dataframe(
        value=display_tasks(),
        label="當前任務列表 (從 Google Sheet 載入)",
        headers=TABLE_HEADERS,
        interactive=False,
        row_count=(5, 'dynamic'),
        # ***** 這是最終修正：將 'dynamic' 替換為明確的數字 6 *****
        col_count=6
        # ******************************************************
    )

    # ... (介面定義的其餘部分保持不變) ...


    gr.Button("🔄 重新載入最新資料").click(
        fn=handle_reload,
        inputs=None,
        outputs=[output_status_message, task_table]
    )

    with gr.Tab("新增/修改/刪除任務"):
        gr.Markdown("## 新增任務")
        with gr.Row():
            add_name = gr.Textbox(label="任務名稱")
            add_desc = gr.Textbox(label="描述", lines=1)
            est_date = gr.Textbox(label="預計完成日期 (YYYY-MM-DD)", value=datetime.now().strftime('%Y-%m-%d'))
            gr.Button("➕ 新增任務").click(
                fn=handle_add_task,
                inputs=[add_name, add_desc, est_date],
                outputs=[output_status_message, task_table]
            )

        gr.Markdown("## 刪除/更新狀態")
        with gr.Row():
            del_id = gr.Textbox(label="任務 ID (刪除/修改用)")

            with gr.Column(scale=1):
                gr.Button("❌ 刪除任務 (✅ 5)").click(
                    fn=handle_delete_task,
                    inputs=[del_id],
                    outputs=[output_status_message, task_table]
                )
            with gr.Column(scale=1):
                status_choice = gr.Radio(["完成", "待辦"], label="設定新狀態", value="完成")
                gr.Button("✍️ 更新任務狀態 (✅ 6.1)").click(
                    fn=handle_toggle_status,
                    inputs=[del_id, status_choice],
                    outputs=[output_status_message, task_table]
                )

    with gr.Tab("查詢與統計"):
        gr.Markdown("## 任務查詢 (✅ 6.2: 時間與狀態)")
        with gr.Row():
            query_status = gr.Radio(["All", "Completed", "Pending"], label="按狀態查詢", value="All")
            query_start_date = gr.Textbox(label="起始日期 (YYYY-MM-DD)", placeholder="留空則不限制")
            query_end_date = gr.Textbox(label="結束日期 (YYYY-MM-DD)", placeholder="留空則不限制")

        query_button = gr.Button("🔍 執行查詢")
        query_output_table = gr.Dataframe(label="查詢結果", interactive=False, row_count=(5, 'dynamic'))

        query_button.click(
            fn=handle_query_tasks,
            inputs=[query_status, query_start_date, query_end_date],
            outputs=[query_output_table]
        )

        gr.Markdown("## 任務統計圖 (✅ 4)")
        visualization_output = gr.Image(label="任務狀態分佈圖", type="filepath")
        gr.Button("📊 生成統計圖").click(
            fn=handle_visualize,
            inputs=None,
            outputs=visualization_output
        )

    with gr.Tab("檔案 I/O (CSV/JSON)"):
        gr.Markdown("## 匯入與匯出紀錄 (✅ 3)")

        with gr.Row():
            file_action = gr.Radio(["匯出", "匯入"], label="操作類型", value="匯出")
            file_format = gr.Radio(["CSV", "JSON"], label="檔案格式", value="CSV")

        file_path_relative = gr.Textbox(
            label="檔案名稱 (會儲存在 Google Drive 專案資料夾中)",
            value="records.csv"
        )

        gr.Button("💾 執行檔案 I/O").click(
            fn=handle_file_io,
            inputs=[file_action, file_path_relative, file_format],
            outputs=[output_status_message, task_table]
        )

Google Sheet 連線成功。


In [ ]:
# 請執行此 Cell 來檢查檔案是否存在
import os

# 這是您在 Cell 2 中設定的路徑
check_path = '/content/drive/MyDrive/Task_Tracker_Project/service_account.json'

if os.path.exists(check_path):
    print(f"✅ 檔案確認存在於: {check_path}")
else:
    print(f"❌ 檔案不存在。請檢查以下幾點:")
    print(f"   1. 請檢查您的 Google Drive 中是否有此檔案。")
    print(f"   2. 請檢查檔案名稱是否 EXACTLY 是 service_account.json (沒有多餘空格或錯誤大小寫)。")

    # 嘗試列出資料夾內容 (幫助您手動檢查)
    try:
        folder_path = '/content/drive/MyDrive/Task_Tracker_Project/'
        print("\n📁 資料夾內容:")
        print(os.listdir(folder_path))
    except FileNotFoundError:
         print("\n🚨 專案資料夾不存在，請重新執行 Cell 2 確認路徑。")

✅ 檔案確認存在於: /content/drive/MyDrive/Task_Tracker_Project/service_account.json


In [ ]:
# Cell 7: 運行 Gradio 介面
if __name__ == "__main__":
    print("啟動 Gradio 介面，請點擊下方 Public URL 連結...")
    # share=True 會生成一個可公開存取的連結，適合 Colab 環境
    demo.launch(share=True)

啟動 Gradio 介面，請點擊下方 Public URL 連結...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd608004ae778ff401.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
